# DDPG - Continuous Control

---

In this notebook, we will train an agent to navigate the Unity ML-Agents Reacher environment, which in this version contains 20 identical agents, each with its own copy of the environment. The goal is to train a double-jointed arm to reach different target locations. The training algorithm that is used for training is DDPG, implemented in PyTorch.

### 1. Import Required Packages

We begin by importing some necessary packages. If the code cell below returns an error, please revisit the project instructions on the README.md file.

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import Agent
import torch

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

We set the hyperparameters we will use later during the DDPG implementation. The values are chosen using trial and error and from RL baselines

In [ ]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 1e-4               # learning rate of the actor and of the critic
WEIGHT_DECAY = 0.       # L2 weight decay

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

As mentioned earlier, in this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of our agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Train Agent

The code below initiates the agents and starts the training loop. Once the average reward over 100 episodes reaches 30, the training stops and the weights are saved in  **_checkpoint\_reacher\_critic.pth_** for the actor network and  **_checkpoint\_reacher\_critic.pth_** for the critic network.

In [ ]:
# initialise agents
agents = Agent(
    state_size=state_size, 
    action_size=action_size, 
    num_agents=num_agents, 
    random_seed=0,
    buffer_size = BUFFER_SIZE,
    batch_size = BATCH_SIZE,
    gamma = GAMMA,
    tau = TAU,
    lr = LR,
    weight_decay = WEIGHT_DECAY
)

In [ ]:
def ddpg_train(n_episodes=1000):

    total_scores = []                                      # list containing scores from each episode
    scores_window = deque(maxlen=100)                      # last 100 scores
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations              # get the current state (for each agent)
        scores = np.zeros(num_agents)                      # initialize the score (for each agent)
        while True:
            actions = agents.act(states)                   # select an action (for each agent)
            env_info = env.step(actions)[brain_name]       # send all actions to tne environment
            next_states = env_info.vector_observations     # get next state (for each agent)
            rewards = env_info.rewards                     # get reward (for each agent)
            done = env_info.local_done                     # see if episode has finished
            
            agents.step(states, actions, rewards, next_states, done)
            scores += np.mean(rewards)
            states = next_states
            if np.any(done): 
                break 
        scores_window.append(scores)       # save most recent score
        total_scores.append(scores)        # save scores without deque to plot later
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=30:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agents.actor_local.state_dict(), 'checkpoint_reacher_actor.pth')
            torch.save(agents.critic_local.state_dict(), 'checkpoint_reacher_critic.pth')
            break
    return total_scores

scores = ddpg_train()

### 4. Plot Scores

Once training has finished we can plot the average reward across the 20 agents.

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Test Trained Agent on Environment
Once this cell is executed, we can watch the agent's performance for one episode.

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]            # reset the environment    
states = env_info.vector_observations                         # get the current state (for each agent)
scores = np.zeros(num_agents)                                 # initialize the score (for each agent)
while True:
    actions = agents.act(states)                               # select an action (for each agent)
    env_info = env.step(actions)[brain_name]                   # send all actions to tne environment
    next_states = env_info.vector_observations                 # get next state (for each agent)
    rewards = env_info.rewards                                 # get reward (for each agent)
    dones = env_info.local_done                                # see if episode finished
    scores += env_info.rewards                                 # update the score (for each agent)
    states = next_states                                       # roll over states to next time step
    if np.any(dones):                                          # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 6. Load Saved Weights and Test on Environment
Load the checkpoint weights to check the saved agent's performance for one episode.

In [ ]:

# load the weights from file
agents.actor_local.load_state_dict(torch.load('checkpoint_reacher_actor.pth'))
agents.critic_local.load_state_dict(torch.load('checkpoint_reacher_critic.pth'))

env_info = env.reset(train_mode=False)[brain_name]            # reset the environment    
states = env_info.vector_observations                         # get the current state (for each agent)
scores = np.zeros(num_agents)                                 # initialize the score (for each agent)
while True:
    actions = agents.act(states)                               # select an action (for each agent)
    env_info = env.step(actions)[brain_name]                   # send all actions to tne environment
    next_states = env_info.vector_observations                 # get next state (for each agent)
    rewards = env_info.rewards                                 # get reward (for each agent)
    dones = env_info.local_done                                # see if episode finished
    scores += env_info.rewards                                 # update the score (for each agent)
    states = next_states                                       # roll over states to next time step
    if np.any(dones):                                          # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))  

When finished, you can close the environment.

In [ ]:
env.close()